In [2]:
import numpy as np
import scipy as sp

In [3]:
# Generate random real nxn block hermitian matrix with block size b
def randHerm(n, b = 1):
    m = n*b
    A = np.random.rand(m, m)
    for i in np.arange(m):
        for j in range(i+1, m):
            A[i, j] = A[j, i]
    return A

In [4]:
def randVec(n, b = 1):
    return np.random.rand(n*b, b)

In [214]:
def block_lanczos(H,V,q,reorth = 0):
    
#     H is dxd, V is d x b, b is block size
#     @ is normal matrix multiplication
#     q is the number of iterations
    Z = np.copy(V)
    d,b = Z.shape
    
    M = [np.zeros((b,b),dtype=H.dtype)] * q
    B = [np.zeros((b,b),dtype=H.dtype)] * (q + 1)
    
    Q = np.zeros((d,b*(q+1)),dtype=H.dtype)

#     if not assume orthogonal then use qr to orthogonalize 
    Q[:,0:b],B[0] = np.linalg.qr(Z)
    for k in range(0,q):
        
#         Qk is the next column of blocks
        Qk = Q[:,k*b:(k+1)*b]
#         Q_k-1 = 0 when k = 0, so Z = H@Qk.
        if k > 0:
            Qkm1 = Q[:,(k-1)*b:k*b]
        else:
            Qkm1 = np.zeros((q, b))
            
        Z = H @ Qk - Qkm1 @ (B[k].conj().T) if k > 0 else H @ Qk
        
        if reorth > k:
            Z -= Q[:,:k * b] @ (Q[:,:k * b].conj().T @ Z)
            Z -= Q[:,:k * b] @ (Q[:,:k * b].conj().T @ Z)

        M[k] = Qk.conj().T @ Z
        Z -= Qk @ M[k]
        
        Q[:,(k + 1) * b:(k + 2) * b],B[k + 1] = np.linalg.qr(Z)
    
    Qk = Q[:, :b*q]
    Qkp1 = Q[:, b*q:]

    return Qk, Qkp1, M, B

In [215]:
def get_block_tridiag(M,R):

    q = len(M)
    b = len(M[0])
    
    T = np.zeros((q*b,q*b),dtype=M[0].dtype)

    for k in range(q):
        T[k*b:(k+1)*b,k*b:(k+1)*b] = M[k]

    for k in range(q-1):
        T[(k+1)*b:(k+2)*b,k*b:(k+1)*b] = R[k]
        T[k*b:(k+1)*b,(k+1)*b:(k+2)*b] = R[k].conj().T
    
#     following for loop above. yet to check if this is corrrect, but at least its bottom triag.
    bkp1 = R[q]
    return T, bkp1

In [216]:
# n is size of block matrix, d block size, i is the position of the identity matrix
def Ei(n, d, i):
    Ei = np.zeros((n*d, d))
    Ei[(i-1)*d:i*d, :] = np.identity(d)
    return Ei

2.3454780933085958e-15

In [229]:
# test output of Lanczos algorithm
n = 5
b = 2
k = 5

H = randHerm(n, b)
V = randVec(n, b)
Qk, Qkp1, M, B = block_lanczos(H, V, k, 1000)
T, bkp1 = get_block_tridiag(M,B)

if (np.linalg.norm(Qk.T@Qk-np.identity(np.shape(Qk.T@Qk)[0])) <= 1e-8):
    print("Qk*Qk = I test passed")

Qk@T + Qkp1@bkp1@Ei(n, b, k).conj().T-H@Qk

Qk*Qk = I test passed'


array([[ 1.59326744e-01,  9.47228898e-01,  3.05180900e-01,
        -2.44087093e-01,  6.58069409e-01,  5.64464383e-01,
         5.40948982e-02,  5.72148308e-03, -8.65765826e-03,
         4.29982411e-02],
       [ 1.12702770e-02, -6.27762870e-02,  8.02435002e-01,
        -1.02223370e-01, -2.31709011e-01, -3.78324878e-01,
         2.50727024e-02,  1.10578681e-03,  1.02242141e-02,
        -3.18178219e-02],
       [ 5.96020190e-02,  1.99242734e-01,  1.91846378e-01,
         2.64030909e-01, -5.07887829e-02, -1.61291431e-01,
        -2.17578292e-01, -3.95997564e-01,  5.21570859e-04,
        -7.69548481e-03],
       [-3.21765954e-01, -1.31011176e+00,  3.32678595e-01,
         8.79524493e-02, -1.49816602e-01,  6.38690839e-01,
        -9.74331969e-02, -2.42085050e-01, -9.69504044e-03,
        -3.83830998e-02],
       [-4.64028176e-02,  8.38273092e-02,  1.01214031e-01,
        -3.80177561e-01,  4.19537573e-01,  1.06152008e-01,
         1.28674476e-01,  3.11506088e-01,  2.18115242e-02,
         7.

In [227]:
np.linalg.norm(Qk.T@Qk-np.identity(np.shape(Qk.T@Qk)[0]))

2.3454780933085958e-15